In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:


mask_rcnn_pred_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/mask_rcnn/results/inference/inf1.bbox.json"
faster_rcnn_pred_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/faster_rcnn/inference/inf1.bbox.json"
rtm_det_pred_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/rtm_det/inference/inf1.bbox.json"

In [4]:
import os
import json
import argparse
from PIL import Image, ImageOps
import cv2
import numpy as np
from pycocotools import mask
import pandas as pd
from glob import glob
dataset_path = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master"

annotation_path = os.path.join(dataset_path, "master_dataset.json")

with open(annotation_path, 'r', encoding='utf8') as fin:
    orig_annotation = json.load(fin)

images = orig_annotation["images"]
annotations = orig_annotation["annotations"]
categories = orig_annotation["categories"]

def drawline(img,pt1,pt2,color,thickness=1,style='dotted',gap=10):
    dist =((pt1[0]-pt2[0])**2+(pt1[1]-pt2[1])**2)**.5
    pts= []
    for i in  np.arange(0,dist,gap):
        r=i/dist
        x=int((pt1[0]*(1-r)+pt2[0]*r)+.5)
        y=int((pt1[1]*(1-r)+pt2[1]*r)+.5)
        p = (x,y)
        pts.append(p)

    if style=='dotted':
        for p in pts:
            cv2.circle(img,p,thickness,color,-1)
    else:
        s=pts[0]
        e=pts[0]
        i=0
        for p in pts:
            s=e
            e=p
            if i%2==1:
                cv2.line(img,s,e,color,thickness)
            i+=1

def drawpoly(img,pts,color,thickness=1,style='dotted',):
    s=pts[0]
    e=pts[0]
    pts.append(pts.pop(0))
    for p in pts:
        s=e
        e=p
        drawline(img,s,e,color,thickness,style)

def drawrect(img,pt1,pt2,color,thickness=1,style='dotted'):
    pts = [pt1,(pt2[0],pt1[1]),pt2,(pt1[0],pt2[1])]
    drawpoly(img,pts,color,thickness,style)

class BoundingBox(object):
    def __init__(self, x1, y1, x2, y2, label=None, score=1, counts = None):
        self.x1 = min(x1, x2)
        self.x2 = max(x1, x2)
        self.y1 = min(y1, y2)
        self.y2 = max(y1, y2)
        self.label = label
        self.score = score
        #self.counts = counts

    def get_coordinate_as_list(self):
        return [self.x1, self.y1, self.x2, self.y2]

    def __repr__(self):
        return f"{self.x1} {self.y1} {self.x2} {self.y2} {self.label}"

    def refined_get_iou(bb1, bb2):
      boxA= bb1.get_coordinate_as_list()
      boxB= bb2.get_coordinate_as_list()
      xA = max(boxA[0], boxB[0])
      yA = max(boxA[1], boxB[1])
      xB = min(boxA[2], boxB[2])
      yB = min(boxA[3], boxB[3])

      # compute the area of intersection rectangle
      interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

      # compute the area of both the prediction and ground-truth rectangles
      boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
      boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

      # compute the intersection over union by taking the intersection
      # area and dividing it by the sum of prediction + ground-truth
      # areas - the interesection area
      iou = interArea / float(boxAArea + boxBArea - interArea)

      # return the intersection over union value
      return iou

    @staticmethod
    def get_iou(bb1, bb2):
        """
        Calculate the Intersection over Union (IoU) of two bouding boxes

        Parameters
        -----------
        bb1 : list [x1, y1, x2, y2] or boundingbox Object

        bb2 : list [x1, y1, x2, y2] or boundingbox Object

        Returns
        --------
        float
            in [0, 1]
        """
        if isinstance(bb1, BoundingBox):
            bb1= bb1.get_coordinate_as_list()
        if isinstance(bb2, BoundingBox):
            bb2= bb2.get_coordinate_as_list()
        #bb1 = [int(x) for x in bb1]
        #bb2 = [int(x) for x in bb2]
        assert bb1[0] <= bb1[2]
        assert bb1[1] <= bb1[3]
        assert bb2[0] <= bb2[2]
        assert bb2[1] <= bb2[3]
        # determine the coordinate od the intersection rectangle
        x_left = max(bb1[0], bb2[0])
        y_top = max(bb1[1], bb2[1])
        x_right = min(bb1[2], bb2[2])
        y_bottom = min(bb1[3], bb2[3])

        if x_right < x_left or y_bottom < y_top:
            return 0.0

        intersection_area = (x_right - x_left)*(y_bottom - y_top)
        bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
        bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])

        iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
        assert iou >= 0.0
        assert iou <= 1.0
        return iou

class Example(object):

    def __init__(self, id=0, filename=None, gt_bb=[], pred_bb=[]):
        self.id=id
        self.filename = filename
        self.gt_bb = gt_bb
        self.pred_bb = pred_bb

    def add_gt_bb(self,bb):
        self.gt_bb.append(bb)

    def add_pred_bb(self, bb):
        self.pred_bb.append(bb)



def calculate_f1(examples, score_thr, iou_thr):

    glb_tp = 0
    glb_fp = 0
    glb_fn = 0
    mean_precision = 0
    mean_recall = 0
    mean_f1 = 0
    wrong = []
    count = 0
    error = 0

    label_metrics = {}
    cat_name = [cat["name"] for cat in categories]
    print("cat len and names",len(cat_name),cat_name)
    for cat in cat_name:
      if cat not in list(label_metrics.keys()):
          label_metrics[cat] = {"tp":0,"fp":0,"fn":0,"count":0}

    wrong_ins = []

    for example in examples:

        tp, fp, fn= 0, 0, 0
        gt_bbs = example.gt_bb
        pred_bbs = [x for x in example.pred_bb if x.score > score_thr]
        tmp = len(pred_bbs)
        if gt_bbs:
          for gt_bb in gt_bbs:

              pred_ok = []
              label_metrics[gt_bb.label]["count"]+=1
              for pred_bb in pred_bbs:
                  #print("pred_bb.label",pred_bb.label)
                  if pred_bb.label == gt_bb.label and BoundingBox.refined_get_iou(gt_bb, pred_bb) > iou_thr:
                      pred_ok.append(pred_bb)
              if pred_ok == []:
                  fn+=1
                  label_metrics[gt_bb.label]["fn"]+=1
              else:
                  tp+=1
                  label_metrics[gt_bb.label]["tp"]+=1
                  for elem in pred_ok:
                      pred_bbs.remove(elem)
          fp = len(pred_bbs)
          for neg_prbb in pred_bbs:
              label_metrics[neg_prbb.label]["fp"]+=1

        else:
          count += 1
          fp = len(pred_bbs)
          error += 1

        if len(pred_bbs) > 0:
          wrong.append(example.filename)
          wrong_ins.append(example)

        #print(f"{example.filename}\t\ttp: {tp}\tfp: {fp}\tfn: {fn} \t {str(pred_bbs)}")
        precision = tp/(tp+fp) if tp>0 else 0
        recall = tp/(tp+fn) if tp>0 else 0

        mean_precision+=precision
        mean_recall+=recall
        mean_f1+= (2*(precision*recall))/(precision+recall) if tp > 0 else 0
        glb_tp += tp
        glb_fp += fp
        glb_fn += fn
    glb_precision = glb_tp/(glb_tp+glb_fp) if glb_tp > 0 else 0
    glb_recall = glb_tp/(glb_tp+glb_fn) if glb_tp > 0 else 0
    glb_f1 = (2*(glb_precision*glb_recall))/(glb_precision+glb_recall) if glb_tp > 0 else 0
    mean_precision /=len(examples)
    mean_recall /= len(examples)
    mean_f1 /= len(examples)
    labelwise_metrics = {}
    for lb_name, met in label_metrics.items():
        tp = met["tp"]
        fp = met["fp"]
        fn = met["fn"]
        precision = tp/(tp+fp) if tp>0 else 0
        recall = tp/(tp+fn) if tp>0 else 0
        mean_f1+= (2*(precision*recall))/(precision+recall) if tp > 0 else 0
        label_metrics[lb_name]["precision"] = precision
        label_metrics[lb_name]["recall"] = recall
        label_metrics[lb_name]["mean_f1"] = mean_f1



    print("negatives: ", count)
    print("error: ", error)
    print(f"glb_tp={glb_tp}")
    print(f"glb_fp={glb_fp}")
    print(f"glb_fn={glb_fn}")
    return {
        "mean_precision":mean_precision,
        "mean_recall":mean_recall,
        "mean_F1": mean_f1,
        "global_precision": glb_precision,
        "global_recall": glb_recall,
        "global_f1": glb_f1
    }, wrong,wrong_ins,label_metrics
def draw_bb(examples, args):
    os.makedirs('draw_bb',exist_ok=True)
    file = open('bbs.txt','w',encoding='utf8')
    for example in examples:
        file.write(f'{example.filename}\n\n')
        gt_bbs = example.gt_bb
        img_path = os.path.join(args.img_dir,example.filename).replace('/','\\')
        pred_bbs = [x for x in example.pred_bb if x.score > args.score_thr]
        img = cv2.imread(img_path)
        #img = np.array(Image.open(img_path))

        for gt_bb in gt_bbs:
            box = [int(x) for x in gt_bb.get_coordinate_as_list()]
            file.write(str(gt_bb.get_coordinate_as_list())+'\n')
            cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), 2)
            pred_ok = []
            for pred_bb in pred_bbs:
                if pred_bb.label == gt_bb.label and BoundingBox.get_iou(gt_bb, pred_bb) > args.iou_thr:
                    pred_ok.append(pred_bb)
            for pred in pred_ok:
                box = [int(x) for x in pred.get_coordinate_as_list()]

                iou_val = BoundingBox.get_iou(gt_bb, pred)

                #text = "score:{:.2f} iou:{:.2f} {}".format(pred.score, iou_val, pred.label)
                text = "{}".format(pred.label)
                cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
                #drawrect(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
                cv2.putText(img, text, (box[0], box[1]), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0,0,0), 1)
        file.write('\n')
        out_path = os.path.join('draw_bb',example.filename).replace("/","\\")
        cv2.imwrite(out_path,img)
    file.close()

def mask_to_polygons(mask):
        # cv2.RETR_CCOMP flag retrieves all the contours and arranges them to a 2-level
        # hierarchy. External contours (boundary) of the object are placed in hierarchy-1.
        # Internal contours (holes) are placed in hierarchy-2.
        # cv2.CHAIN_APPROX_NONE flag gets vertices of polygons from contours.
    mask = np.ascontiguousarray(mask)  # some versions of cv2 does not support incontiguous arr
    res = cv2.findContours(mask.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    hierarchy = res[-1]
    if hierarchy is None:  # empty mask
        return [], False
    has_holes = (hierarchy.reshape(-1, 4)[:, 3] >= 0).sum() > 0
    res = res[-2]
    res = [x.flatten() for x in res]
        # These coordinates from OpenCV are integers in range [0, W-1 or H-1].
        # We add 0.5 to turn them into real-value coordinate space. A better solution
        # would be to first +0.5 and then dilate the returned polygon by 0.5.
    res = [x + 0.5 for x in res if len(x) >= 6]
    return res, has_holes

def draw_bb1(examples, args, out_dir):
    os.makedirs(out_dir,exist_ok=True)
    #file = open('bbs.txt','w',encoding='utf8')
    for example in examples:
        #file.write(f'{example.filename}\n\n')
        gt_bbs = example.gt_bb
        #print("path: ", args.img_dir)
        img_path = os.path.join(args.img_dir,example.filename)#.replace('/','\\')
        pred_bbs = [x for x in example.pred_bb if x.score > args.score_thr]
        #img = cv2.imread(img_path)
        img = Image.open(img_path).convert('RGB')
        img = ImageOps.exif_transpose(img)
        imageWidth, imageHeight = img.size
        img = np.array(img)
        img = img[:, :, ::-1].copy()

        for gt_bb in gt_bbs:
            box = [int(x) for x in gt_bb.get_coordinate_as_list()]
            #file.write(str(gt_bb.get_coordinate_as_list())+'\n')
            cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), 2)

        for pred in pred_bbs:
            box = [int(x) for x in pred.get_coordinate_as_list()]

            #iou_val = BoundingBox.get_iou(gt_bb, pred)

                #text = "score:{:.2f} iou:{:.2f} {}".format(pred.score, iou_val, pred.label)
            text = "{}".format(pred.label)
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)


            '''
            counts, _ = mask_to_polygons(pred.counts)
            if len(counts) == 0:
              continue
            counts = counts[0].astype(np.int32)
            counts = counts.reshape((-1,1,2))
            #print(counts)
            cv2.polylines(img,counts,True,(0,255,0),3)
            '''
            '''
            mask = pred.counts
            mask = np.ascontiguousarray(mask)  # some versions of cv2 does not support incontiguous arr
            res,_ = cv2.findContours(mask.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            cv2.drawContours(img, res, -1, (0,255,0), 3)
            '''


            fontScale = 1.5

            if imageWidth <= 500:
              fontScale = 0.4
            elif imageWidth <= 700:
              fontScale = 0.55
            elif imageWidth <= 1050:
              fontScale = 0.75
            elif imageWidth <= 1700:
              fontScale = 1
            elif imageWidth >= 3000:
              fontScale = 1.8

                #drawrect(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
            cv2.putText(img, text, (box[2], box[3]), cv2.FONT_HERSHEY_COMPLEX, fontScale, (0,0,0), 2)
        #file.write('\n')
        if len(gt_bbs) + len(pred_bbs) > 0:
            out_path = os.path.join(out_dir,example.filename).replace('scartati/','')
            cv2.imwrite(out_path,img)
        #img = Image.fromarray(img)
        #img.save(out_path)
def draw_bb2(examples, args, out_dir):
    os.makedirs(out_dir,exist_ok=True)
    #file = open('bbs.txt','w',encoding='utf8')
    for example in examples:
        #file.write(f'{example.filename}\n\n')
        gt_bbs = example.gt_bb
        #print("path: ", args.img_dir)
        img_path = os.path.join(args.img_dir,example.filename)#.replace('/','\\')
        pred_bbs = [x for x in example.pred_bb if x.score > args.score_thr]
        #img = cv2.imread(img_path)
        img = Image.open(img_path).convert('RGB')
        img = ImageOps.exif_transpose(img)
        imageWidth, imageHeight = img.size
        img = np.array(img)
        img = img[:, :, ::-1].copy()

        for gt_bb in gt_bbs:
            box = [int(x) for x in gt_bb.get_coordinate_as_list()]
            #file.write(str(gt_bb.get_coordinate_as_list())+'\n')
            cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), 2)

        for pred in pred_bbs:
            box = [int(x) for x in pred.get_coordinate_as_list()]

            #iou_val = BoundingBox.get_iou(gt_bb, pred)

                #text = "score:{:.2f} iou:{:.2f} {}".format(pred.score, iou_val, pred.label)
            text = "{}".format(pred.label)
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)


            '''
            counts, _ = mask_to_polygons(pred.counts)
            if len(counts) == 0:
              continue
            counts = counts[0].astype(np.int32)
            counts = counts.reshape((-1,1,2))
            #print(counts)
            cv2.polylines(img,counts,True,(0,255,0),3)
            '''
            '''
            mask = pred.counts
            mask = np.ascontiguousarray(mask)  # some versions of cv2 does not support incontiguous arr
            res,_ = cv2.findContours(mask.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            cv2.drawContours(img, res, -1, (0,255,0), 3)
            '''


            fontScale = 0.5

            if imageWidth <= 500:
              fontScale = 0.3
            elif imageWidth <= 700:
              fontScale = 0.44
            elif imageWidth <= 1050:
              fontScale = 0.66
            elif imageWidth <= 1700:
              fontScale = 0.5
            elif imageWidth >= 3000:
              fontScale = 0.9

                #drawrect(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
            cv2.putText(img, text, (box[2], box[3]), cv2.FONT_HERSHEY_COMPLEX, fontScale, (0,0,0), 2)
        #file.write('\n')
        if len(gt_bbs) + len(pred_bbs) > 0:
            out_path = os.path.join(out_dir,example.filename).replace('scartati/','')
            cv2.imwrite(out_path,img)
        #img = Image.fromarray(img)
        #img.save(out_path)
def main(args):
    labels_map = {}
    examples = []
    with open(args.test_json, encoding='utf8') as f:
        test_json_data = json.load(f)
    with open(args.prediction_json, encoding='utf8') as f:
        pred_json_data = json.load(f)

    for item in test_json_data["categories"]:
        labels_map[item["id"]]=item["name"]

    # if args.remove_negative:
    #     negative_imgs = get_neg()
    #     negative_imgs = [os.path.splitext(os.path.basename(img))[0].split("-")[-1]+".jpg" for img in negative_imgs]
    #     print("neg images",negative_imgs)
    #     test_images = test_json_data["images"]
    #     #pred_images = pred_json_data["images"]
    #     print("test",test_images[0])
    #     print("pred", pred_json_data[0])
    #     test_filt = [img for img in test_images if img["file_name"] not in negative_imgs]
    #     print("prev vs now", len(test_images),len(test_filt))
    #     filt_ids = [img["id"] for img in test_filt]
    #     print("filt_ids",filt_ids[:5])

    #     pred_filt = [img for img in pred_json_data if img["image_id"] in filt_ids ]

    #     test_json_data["images"] = test_filt

    #     pred_json_data = pred_filt

    for imgs in test_json_data["images"]:
        #example = Example(imgs["id"], imgs["file_name"])

        #if imgs["file_name"] in test_source:
          #continue

        item_id = imgs["id"]
        gt_bb = []
        pred_bb = []
        for gt_annotations in test_json_data["annotations"]:

            if gt_annotations["image_id"] == item_id:
                tmp_box = gt_annotations["bbox"]
                tmp_box = [tmp_box[0], tmp_box[1],tmp_box[0]+tmp_box[2], tmp_box[1]+tmp_box[3]]
                #tmp_box = [tmp_box[0], tmp_box[1],tmp_box[0]+tmp_box[2], tmp_box[1]+tmp_box[3]]
                #example.add_gt_bb(BoundingBox(*tmp_box, labels_map[gt_annotations["category_id"]]))
                gt_bb.append(BoundingBox(*tmp_box, labels_map[gt_annotations["category_id"]]))
        for pred_annotations in pred_json_data:

            if pred_annotations["image_id"] == item_id:
                tmp_box = pred_annotations["bbox"]
                tmp_box = [tmp_box[0], tmp_box[1],tmp_box[0]+tmp_box[2], tmp_box[1]+tmp_box[3]]
                #example.add_pred_bb(BoundingBox(*tmp_box, labels_map[pred_annotations["category_id"]], score=pred_annotations["score"]))
                pred_bb.append(BoundingBox(*tmp_box, labels_map[pred_annotations["category_id"]], score=pred_annotations["score"]))#, counts=mask.decode(pred_annotations["segmentation"])))
        #print(f"{len(gt_bb)}")
        examples.append(Example(item_id,imgs["file_name"], gt_bb, pred_bb))


    """    if args.draw_bb:
          out_dir_all = args.out_dir
          draw_bb1(examples,args, out_dir_all)"""


    F1_val, wrong,wrong_ins,label_metrics = calculate_f1(examples, args.score_thr, args.iou_thr)
    print("example",examples)
    print("wrong.. ", wrong)
    with open((os.path.join(args.out_dir,"wrong.json")), 'w', encoding='utf-8') as f:
      json.dump(wrong, f, ensure_ascii=False, indent=4)

    for key, val in F1_val.items():
        F1_val[key] = round(val*100, 2)

        print(f"{key}: {round(val*100, 2)}")

    print("data length", len(examples))
    if args.draw_bb:
        out_dir_all = args.out_dir
        #draw_bb1(examples,args, out_dir_all)
        out_dir_wrong = args.out_dir + "/wrong"
        if not os.path.exists(out_dir_wrong):
          os.mkdir(out_dir_wrong)
        #draw_bb1(wrong_ins,args, out_dir_wrong)

    cols = []
    data = []
    for key, val in F1_val.items():
        cols.append(key)
        data.append(val)
    if not os.path.exists(os.path.join(args.out_dir,"metrics")):
          os.mkdir(os.path.join(args.out_dir,"metrics"))
    metrics_path = os.path.join(args.out_dir,"metrics")
    #df = pd.DataFrame.from_dict(F1_val, dtype = np.float64, orient='index', columns=['Value'])
    df = pd.DataFrame([data], columns=cols)
    df.to_csv(os.path.join(metrics_path,"metrics.csv"))

    cols = []
    data = []
    for key, val in label_metrics.items():
        cols.append(key)
        data.append(val)
    df = pd.DataFrame.from_dict(label_metrics, orient="index")
    df.to_csv(os.path.join(metrics_path,"label_metrics.csv"))


    return F1_val["mean_F1"]

In [5]:
if __name__ == "__main__":

    conf = 0.3
    iou = 0.9

    parser = argparse.ArgumentParser()
    parser.add_argument("--test_json", default="test.json")
    parser.add_argument("--prediction_json", default="coco_instances_results(7).json")
    parser.add_argument("--score_thr", default=conf, type=float)
    parser.add_argument("--iou_thr", default=iou, type=float)
    parser.add_argument("--draw_bb", action="store_true", default=False)
    parser.add_argument("--img_dir")


    parser.add_argument("--remove_negative", action="store_true", default=True)

    parser.add_argument("--out_dir")


    args = parser.parse_args(args=[])


    args.test_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/test.json"

    args.prediction_json = mask_rcnn_pred_json

    args.img_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/images"


    args.draw_bb =True


    args.out_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/mask_rcnn/results/inference/test_plots_score_thr={}_iou_thr={}".format(conf,iou)

    if not os.path.exists(args.out_dir):
      os.mkdir(args.out_dir)
    main(args)


cat len and names 52 ['Address', 'ActionTaken', 'CompName', 'DOB', 'Year', 'ReceivedDate', 'Sections', 'FHName', 'OOODay', 'Date', 'ReceivedTime', 'SusDetails', 'OtherActs', 'GDTime', 'GDNumber', 'OffDirected', 'FIRContents', 'FirNum', 'CompAddress', 'Subdivision', 'GDEntNum', 'Nationality', 'PoliceStation', 'Dist', 'Act', 'OfficerSignature', 'OfficerRank', 'OfficerName', 'Time', 'DirAndDist', 'InspectorSignature', 'OOODate', 'OOOTime', 'OfficerNumber', 'Unknown Content', 'InfoType', 'StolenProp', 'GDPoliceStation', 'G.D. No', 'PropDetails', 'OccurDate', 'DelayReason', 'TransPS', 'InqRep', 'OutPS', 'OutPSDist', 'CompOccupation', 'DespatchDate', 'ActiontakenDirected', 'GDDate', 'CompPassportNo', 'ActiontakenRegistered']
negatives:  0
error:  0
glb_tp=775
glb_fp=3436
glb_fn=3250
example [<__main__.Example object at 0x7b44905d09d0>, <__main__.Example object at 0x7b44905aa9e0>, <__main__.Example object at 0x7b44905a8700>, <__main__.Example object at 0x7b44905b0fa0>, <__main__.Example objec

In [6]:
if __name__ == "__main__":

    conf = 0.3
    iou = 0.8

    parser = argparse.ArgumentParser()
    parser.add_argument("--test_json", default="test.json")
    parser.add_argument("--prediction_json", default="coco_instances_results(7).json")
    parser.add_argument("--score_thr", default=conf, type=float)
    parser.add_argument("--iou_thr", default=iou, type=float)
    parser.add_argument("--draw_bb", action="store_true", default=False)
    parser.add_argument("--img_dir")


    parser.add_argument("--remove_negative", action="store_true", default=True)

    parser.add_argument("--out_dir")


    args = parser.parse_args(args=[])


    args.test_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/test.json"

    args.prediction_json = mask_rcnn_pred_json

    args.img_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/images"


    args.draw_bb =True


    args.out_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/mask_rcnn/results/inference/test_plots_score_thr={}_iou_thr={}".format(conf,iou)

    if not os.path.exists(args.out_dir):
      os.mkdir(args.out_dir)
    main(args)


cat len and names 52 ['Address', 'ActionTaken', 'CompName', 'DOB', 'Year', 'ReceivedDate', 'Sections', 'FHName', 'OOODay', 'Date', 'ReceivedTime', 'SusDetails', 'OtherActs', 'GDTime', 'GDNumber', 'OffDirected', 'FIRContents', 'FirNum', 'CompAddress', 'Subdivision', 'GDEntNum', 'Nationality', 'PoliceStation', 'Dist', 'Act', 'OfficerSignature', 'OfficerRank', 'OfficerName', 'Time', 'DirAndDist', 'InspectorSignature', 'OOODate', 'OOOTime', 'OfficerNumber', 'Unknown Content', 'InfoType', 'StolenProp', 'GDPoliceStation', 'G.D. No', 'PropDetails', 'OccurDate', 'DelayReason', 'TransPS', 'InqRep', 'OutPS', 'OutPSDist', 'CompOccupation', 'DespatchDate', 'ActiontakenDirected', 'GDDate', 'CompPassportNo', 'ActiontakenRegistered']
negatives:  0
error:  0
glb_tp=2774
glb_fp=1437
glb_fn=1251
example [<__main__.Example object at 0x7b44905b0b80>, <__main__.Example object at 0x7b44905b1150>, <__main__.Example object at 0x7b44905a0ca0>, <__main__.Example object at 0x7b44905a11e0>, <__main__.Example obje

In [7]:
if __name__ == "__main__":

    conf = 0.3
    iou = 0.7

    parser = argparse.ArgumentParser()
    parser.add_argument("--test_json", default="test.json")
    parser.add_argument("--prediction_json", default="coco_instances_results(7).json")
    parser.add_argument("--score_thr", default=conf, type=float)
    parser.add_argument("--iou_thr", default=iou, type=float)
    parser.add_argument("--draw_bb", action="store_true", default=False)
    parser.add_argument("--img_dir")


    parser.add_argument("--remove_negative", action="store_true", default=True)

    parser.add_argument("--out_dir")


    args = parser.parse_args(args=[])


    args.test_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/test.json"

    args.prediction_json = mask_rcnn_pred_json

    args.img_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/images"


    args.draw_bb =True


    args.out_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/mask_rcnn/results/inference/test_plots_score_thr={}_iou_thr={}".format(conf,iou)

    if not os.path.exists(args.out_dir):
      os.mkdir(args.out_dir)
    main(args)


cat len and names 52 ['Address', 'ActionTaken', 'CompName', 'DOB', 'Year', 'ReceivedDate', 'Sections', 'FHName', 'OOODay', 'Date', 'ReceivedTime', 'SusDetails', 'OtherActs', 'GDTime', 'GDNumber', 'OffDirected', 'FIRContents', 'FirNum', 'CompAddress', 'Subdivision', 'GDEntNum', 'Nationality', 'PoliceStation', 'Dist', 'Act', 'OfficerSignature', 'OfficerRank', 'OfficerName', 'Time', 'DirAndDist', 'InspectorSignature', 'OOODate', 'OOOTime', 'OfficerNumber', 'Unknown Content', 'InfoType', 'StolenProp', 'GDPoliceStation', 'G.D. No', 'PropDetails', 'OccurDate', 'DelayReason', 'TransPS', 'InqRep', 'OutPS', 'OutPSDist', 'CompOccupation', 'DespatchDate', 'ActiontakenDirected', 'GDDate', 'CompPassportNo', 'ActiontakenRegistered']
negatives:  0
error:  0
glb_tp=3486
glb_fp=725
glb_fn=539
example [<__main__.Example object at 0x7b44905a3640>, <__main__.Example object at 0x7b44905a07f0>, <__main__.Example object at 0x7b4490590df0>, <__main__.Example object at 0x7b4490591f00>, <__main__.Example object

In [8]:
if __name__ == "__main__":

    conf = 0.3
    iou = 0.6

    parser = argparse.ArgumentParser()
    parser.add_argument("--test_json", default="test.json")
    parser.add_argument("--prediction_json", default="coco_instances_results(7).json")
    parser.add_argument("--score_thr", default=conf, type=float)
    parser.add_argument("--iou_thr", default=iou, type=float)
    parser.add_argument("--draw_bb", action="store_true", default=False)
    parser.add_argument("--img_dir")


    parser.add_argument("--remove_negative", action="store_true", default=True)

    parser.add_argument("--out_dir")


    args = parser.parse_args(args=[])


    args.test_json = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/test.json"

    args.prediction_json = mask_rcnn_pred_json

    args.img_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/datasets/master/images"


    args.draw_bb =True


    args.out_dir = "/content/drive/MyDrive/My_PhD_Research/FIR_FORM/models/mask_rcnn/results/inference/test_plots_score_thr={}_iou_thr={}".format(conf,iou)

    if not os.path.exists(args.out_dir):
      os.mkdir(args.out_dir)
    main(args)


cat len and names 52 ['Address', 'ActionTaken', 'CompName', 'DOB', 'Year', 'ReceivedDate', 'Sections', 'FHName', 'OOODay', 'Date', 'ReceivedTime', 'SusDetails', 'OtherActs', 'GDTime', 'GDNumber', 'OffDirected', 'FIRContents', 'FirNum', 'CompAddress', 'Subdivision', 'GDEntNum', 'Nationality', 'PoliceStation', 'Dist', 'Act', 'OfficerSignature', 'OfficerRank', 'OfficerName', 'Time', 'DirAndDist', 'InspectorSignature', 'OOODate', 'OOOTime', 'OfficerNumber', 'Unknown Content', 'InfoType', 'StolenProp', 'GDPoliceStation', 'G.D. No', 'PropDetails', 'OccurDate', 'DelayReason', 'TransPS', 'InqRep', 'OutPS', 'OutPSDist', 'CompOccupation', 'DespatchDate', 'ActiontakenDirected', 'GDDate', 'CompPassportNo', 'ActiontakenRegistered']
negatives:  0
error:  0
glb_tp=3684
glb_fp=523
glb_fn=341
example [<__main__.Example object at 0x7b4490590280>, <__main__.Example object at 0x7b44905be7a0>, <__main__.Example object at 0x7b44905bf4c0>, <__main__.Example object at 0x7b44905bfc70>, <__main__.Example object